## Notebook demonstrujący użycie pozostałych API biblioteki

Inicjalizacja połączenia z serwerem.

In [1]:
import cloudberry.api as cb

cb_port = 9000
cb_config = cb.CloudberryConfig(f'http://localhost:{cb_port}')

In [2]:
# Utils
import math
import datetime
get_time = lambda: math.trunc(datetime.datetime.utcnow().timestamp())

### API klasy Data
Przy pomocy tego API istnieje możliwość wgrania danych do systemu przy pomocy klasy DataPoint, które zawiera pola:
* time - znacznik czasowy
* fields - pola zawierające dane z obliczeń
* tags - znaczniki danych (indeksowane)

In [3]:
# Raw data API
data_api = cb.Data(cb_config)

points = [
    cb.DataPoint(time=get_time(), fields={'version': 0.1}, tags={'origin': 'jupyter_notebook', 'type': 'api_demo'}),
    cb.DataPoint(time=get_time() + 1, fields={'version': 0.2}, tags={'origin': 'jupyter_notebook', 'type': 'api_demo'}),
    cb.DataPoint(time=get_time() + 2, fields={'version': 0.3}, tags={'origin': 'notebook', 'type': 'api_demo'}),
    cb.DataPoint(time=get_time() + 3, fields={'version': 0.4}, tags={'origin': 'notebook', 'type': 'api_demo'}),
]

Zapis danych:

In [4]:
# Save raw data points

data_api.save_data(points)

True

Do pobrania zapisanych danych wykorzystujemy DataFilters, które zawiera pola:
* tags - filtrowanie po wartościach tagów (zalecane)
* tags_presence - filtrowanie po obecności tagów (zalecane)
* fields - filtrowanie po wartości pól

In [5]:
# Query raw data points - by tags values

filters = cb.DataFilters(tags={'origin': 'notebook'}, tags_presence=None, fields=None)
series = data_api.get_data(filters)

series.as_data_frame

,_time,_measurement,origin,type,version,series_name
0,1.604656e+09,cloudberry-default,notebook,api_demo,0.3,raw_data
1,1.604656e+09,cloudberry-default,notebook,api_demo,0.4,raw_data


In [6]:
# Query raw data points - by tags presence, regardless of their values

filters = cb.DataFilters(tags_presence=['origin'])
series = data_api.get_data(filters)

series.as_data_frame

,_time,_measurement,origin,type,version,series_name
0,1.604656e+09,cloudberry-default,jupyter_notebook,api_demo,0.1,raw_data
1,1.604656e+09,cloudberry-default,jupyter_notebook,api_demo,0.2,raw_data
2,1.604656e+09,cloudberry-default,notebook,api_demo,0.3,raw_data
3,1.604656e+09,cloudberry-default,notebook,api_demo,0.4,raw_data


## API klasy Buckets
Do zarządzania sposobem przechowywania danych w bazie TSDB InfluxDB.

In [7]:
# Buckets (InfluxDB-specific) API

buckets_api = cb.Buckets(cb_config)

Wylistowanie wszystkich bucket'ów:

In [8]:
# Get present buckets

buckets_api.get_buckets_names()

['_monitoring',
 '_tasks',
 'cloudberry_logs',
 'cloudberry_metrics',
 'cloudberry_logs_streams']

Tworzenie nowego bucket'a

In [9]:
# Create new bucket

bucket_name = 'wiaderko_danych_0'
buckets_api.create_bucket(bucket_name)

buckets_api.get_buckets_names() # verify

['_monitoring',
 'wiaderko_danych_0',
 '_tasks',
 'cloudberry_logs',
 'cloudberry_metrics',
 'cloudberry_logs_streams']

Usuwanie bucket'a

In [10]:
# Delete new bucket

buckets_api.delete_bucket(bucket_name)

buckets_api.get_buckets_names() # verify

['_monitoring',
 '_tasks',
 'cloudberry_logs',
 'cloudberry_metrics',
 'cloudberry_logs_streams']

## API klasy Query
Do wykonywania query według specyfikacji języka Flux, zwracających pojedynczą serię danych.

In [11]:
# Raw Flux Query (InfluxDB-specific) API

query_api = cb.Query(cb_config)
query_flux = """
from(bucket:"cloudberry_logs")
|> range(start: 0)
|> filter(fn: (r) => r._field == "version")
|> keep(columns: ["_value", "_time"])
"""

query_series = query_api.query_series(query_flux)
query_series.as_data_frame

,result,table,_time,_value,series_name
0,_result,0,1.604656e+09,0.1,flux_query
1,_result,0,1.604656e+09,0.2,flux_query
2,_result,0,1.604656e+09,0.3,flux_query
3,_result,0,1.604656e+09,0.4,flux_query


In [12]:
# Delete created raw data points

data_api.delete_data(filters)

True

## Wgrywanie danych z plików CSV
Aby wgrać pliki CSV z danymi do systemu, należy najpierw utworzyć metadane dla eksperymentu, które opiszą jakiego eksperymentu dotyczą dane oraz konfiguracji, opisujące konfigurację w której uzyskano dany wynik.

In [13]:
# Create metadata necessary for CSV upload - Experiment & Configuration

meta_api = cb.Metadata(cb_config)
experiment_api = meta_api.experiment_api()
configuration_api = meta_api.experiment_configuration_api()

csv_experiment_name = 'Example CSV Experiment'
csv_configuration_name = 'Example CSV Configuration'

# optionally pass parameters to save experiment with
csv_experiment = experiment_api.find_or_create(csv_experiment_name, parameters={'importance': 'high'})
csv_configuration = configuration_api.find_or_create(csv_experiment, csv_configuration_name, parameters={'version': 1})

Do importu danych należy użyć API klasy CsvFileUploader.
W klasie CsvUploadDetails specyfikujemy:
* tags_names - lista kolumn które są tagami i powinny zostać zaindeksowane
* configuration - konfiguracja utworzona wcześniej, opisująca plik
* computation (opcjonalny) - jeżeli podany, dane zostaną przypisane istniejącemu uruchomieniu, w przeciwnym razie zostanie utworzone nowe uruchomienie

In [14]:
# CSV-like Data Upload API

csv_api = cb.CsvFileUploader(cb_config)

csv_file_path = './data/emas_1.csv'
csv_details = cb.CsvUploadDetails(
    tags_names=['WORKPLACE_ID'],
    configuration=csv_configuration,
    computation=None # None for new one, otherwise append data to existing computation 
)

csv_computation = csv_api.upload_file(
    csv_file_path,
    csv_experiment_name,
    csv_details
)

In [15]:
# Verify uploaded CSV data

filters = cb.DataFilters(tags={'WORKPLACE_ID': 0}, computation=csv_computation)
series = data_api.get_data(filters)
series.as_data_frame

,_time,WORKPLACE_ID,_measurement,computationId,ENERGY_SUM,AVERAGE_FITNESS,POPULATION_SIZE,STEP_NUMBER,series_name
0,1.603151e+09,0,cloudberry-default,5fa52a71c0cba37786e8b237,2460.512629,4.598373,98.0,5079.0,raw_data
1,1.603152e+09,0,cloudberry-default,5fa52a71c0cba37786e8b237,2454.008962,4.595291,98.0,10478.0,raw_data
2,1.603153e+09,0,cloudberry-default,5fa52a71c0cba37786e8b237,2529.148288,4.607681,99.0,15843.0,raw_data
3,1.603154e+09,0,cloudberry-default,5fa52a71c0cba37786e8b237,1974.607684,4.608023,76.0,20914.0,raw_data


In [16]:
# Append additional metadata ids to DataSeries

series.meta_ids = meta_api.get_meta_ids(csv_computation)
print(series.meta_ids.__dict__)
series.as_data_frame

{'computation_id': '5fa52a71c0cba37786e8b237', 'configuration_id': '5fa52a71c0cba37786e8b235', 'experiment_id': '5fa52a71c0cba37786e8b234'}


,_time,WORKPLACE_ID,_measurement,computationId,ENERGY_SUM,AVERAGE_FITNESS,POPULATION_SIZE,STEP_NUMBER,series_name,experiment_id,configuration_id,computation_id
0,1.603151e+09,0,cloudberry-default,5fa52a71c0cba37786e8b237,2460.512629,4.598373,98.0,5079.0,raw_data,5fa52a71c0cba37786e8b234,5fa52a71c0cba37786e8b235,5fa52a71c0cba37786e8b237
1,1.603152e+09,0,cloudberry-default,5fa52a71c0cba37786e8b237,2454.008962,4.595291,98.0,10478.0,raw_data,5fa52a71c0cba37786e8b234,5fa52a71c0cba37786e8b235,5fa52a71c0cba37786e8b237
2,1.603153e+09,0,cloudberry-default,5fa52a71c0cba37786e8b237,2529.148288,4.607681,99.0,15843.0,raw_data,5fa52a71c0cba37786e8b234,5fa52a71c0cba37786e8b235,5fa52a71c0cba37786e8b237
3,1.603154e+09,0,cloudberry-default,5fa52a71c0cba37786e8b237,1974.607684,4.608023,76.0,20914.0,raw_data,5fa52a71c0cba37786e8b234,5fa52a71c0cba37786e8b235,5fa52a71c0cba37786e8b237


## API klasy Anomalies
Przy pomocy Anomalies, możemy zbadać wartość wybranego pola dla danego uruchomienia, otrzymując informacje statystyczne o tym polu w obrębie serii.

In [17]:
# # Anomalies Reports API

anomalies_api = cb.Anomalies(cb_config)
report = anomalies_api.get_report(field_name='ENERGY_SUM', computation=csv_computation)

report.__dict__

{'computation_id': '5fa52a71c0cba37786e8b237',
 'stddev': 255.57835624293207,
 'mean': 2354.5693908957446,
 'spread': 554.5406047293914,
 'minimum': 1974.6076836456664,
 'maximum': 2529.1482883750577,
 'max_diff': 554.5406047293914}

In [18]:
# Cleanup
data_api.delete_data(filters)

True

## API klasy ApiConfiguration
Przy pomocy ApiConfiguration możemy modyfikować pewne ustawienia serwera, np. to do jakiego bucket'a w bazie InfluxDB domyślnie trafią wszelkie zbierane dane.

In [19]:
# Configuration Properties API

properties_api = cb.ApiConfiguration(cb_config)

property_key = cb.ApiPropertiesIndex.OVERRIDDEN_DEFAULT_BUCKET_NAME

properties_api.set_property(property_key, "cloudberry_logs_secondary")
property_value = properties_api.get_property(property_key)
print(property_value)

properties_api.delete_property(property_key)

property_value = properties_api.get_property(property_key)
print(property_value == '')

cloudberry_logs_secondary
True
